# Electricity Forecasting - ML Model Development

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

import holidays
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import sklearn.metrics as skm
import statsmodels.api as sm
from sklego.preprocessing import RepeatingBasisFunction
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from statsmodels.tsa.stattools import acf

In [3]:
alt.data_transformers.disable_max_rows()
alt.renderers.set_embed_options(actions=False)

RendererRegistry.enable('default')

## About

This notebook will perform
- ML model training
- ML model evaluation by comparing ML model performance against that of a naive forecast
- assessment of factors identified to be the most important for forecasting electricity consumption

## User Inputs

In [4]:
# Data
# # Date range
start_date = "2015"
end_date = "2020"
# # Source
opsd_data_url = (
    "https://data.open-power-system-data.org/time_series/latest/"
    "time_series_60min_singleindex.csv"
)
# # Column to load
load_col_name_str = "load_actual_entsoe_transparency"

# Features
seasons = {
    "1": "winter",
    "2": "winter",
    "3": "winter",
    "4": "spring",
    "5": "spring",
    "6": "spring",
    "7": "summer",
    "8": "summer",
    "9": "summer",
}

# Country to load
country_names = [
    "DE",
    "CZ",
    "BE",
    "HR",
    "PL",
    "IT",
    "SK",
    "RO",
    "ES",
    "AT",
    "DK",
    "GB_GBN",
]

In [5]:
# columns to load (timestamp column and single country's load column)
raw_mask = [c + f"_{load_col_name_str}" for c in country_names] + ["utc_timestamp"]

holidays_objs = [
    holidays.BE(),
    holidays.CZ(),
    holidays.DE(),
    holidays.HR(),
    holidays.PL(),
    holidays.IT(),
    holidays.SK(),
    holidays.RO(),
    holidays.ES(),
    holidays.AT(),
    holidays.DK(),
    holidays.GB(),
]

# SPLITS - without datetime index
# # full starts on Jan 1 and ends on Sep 30 at 23:00
# # train starts on Jan 1 and ends on Aug 31 at 23:00
# ntr = 243 + 1 + 365
# train, test, full = [
#     slice(None, (ntr * 24) - 1),
#     slice((ntr * 24), ((ntr + 30) * 24) - 1),
#     slice(None, ((ntr + 30) * 24) - 1),
# ]
# SPLITS - with datetime index
train = (slice(None), slice("2018-01-01", "2019-09-01 00:00:00"))
test = (slice(None), slice("2019-09-01", "2019-10-01 00:00:00"))
full = (slice(None), slice("2018-01-01", "2019-10-01 00:00:00"))

In [6]:
class PassthroughTransformer(BaseEstimator, TransformerMixin):
    """passthrough transformer"""

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X


class DFFillNaNMean(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
        self.mean = 0

    def fit(self, X, y=None):
        self.mean = X[self.col].mean()
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        X[self.col] = X[self.col].fillna(self.mean)
        return X

    def fit_transform(self, X, y=None, **kwargs):
        self = self.fit(X, y)
        return self.transform(X)


def fill_target_nans(df, train, test, y_col_name="load"):
    y_train = df.loc[train, y_col_name]
    y_test = df.loc[test, y_col_name]
    pipe_target = DFFillNaNMean(y_col_name)
    _ = pipe_target.fit(y_train.to_frame())
    y_train_filled = pipe_target.transform(y_train.to_frame()).squeeze()
    y_test_filled = pipe_target.transform(y_test.to_frame()).squeeze()
    y = pd.concat([y_train_filled.to_frame(), y_test_filled.to_frame()])  # .squeeze()
    # print(len(X_train), len(y_train_filled), len(X_test), len(y_test_filled))
    # display(y.to_frame())
    return y


def get_coefs_feats(
    pipe_trained, rbf_hourly_feats, numericals, categoricals, X_test, country_name
):
    pp_trained = pipe_trained.named_steps["pre"]

    # Get transformed feature names
    rbf_feat_names = [
        f"rbf_{c}"
        for c in range(1, pp_trained.named_transformers_["nums-rbf-hour"].n_periods + 1)
    ]
    ohe_feat_names = pd.Series(
        pp_trained.named_transformers_["cats"].get_feature_names_out()
    ).str.strip("cats__")

    # handle numericals
    if numericals:
        trans_feat_names = pd.concat(
            [pd.Series(numericals), pd.Series(rbf_feat_names), ohe_feat_names]
        )
        cols_to_transform = numericals + rbf_hourly_feats + categoricals
        float_cols = numericals + rbf_feat_names
        feat_trans = pp_trained.transform(X_test[cols_to_transform])
    else:
        trans_feat_names = pd.concat([pd.Series(rbf_feat_names), ohe_feat_names])
        cols_to_transform = rbf_hourly_feats + categoricals
        float_cols = rbf_feat_names
        feat_trans = pp_trained.transform(X_test[cols_to_transform]).toarray()

    # Get transformed features
    X_test_pre = (
        pd.DataFrame(feat_trans, columns=trans_feat_names)
        .astype({c: pd.Int16Dtype() for c in ohe_feat_names})
        .astype({c: pd.Float32Dtype() for c in float_cols})
    )

    # Get coefficients and scale by std of features
    # https://scikit-learn.org/stable/auto_examples/inspection/
    # plot_linear_model_coefficient_interpretation.html#interpreting-coefficients-scale-matters
    coefs = (
        pipe_trained.named_steps["reg"].regressor_.coef_
        if hasattr(pipe.named_steps["reg"], "regressor_")
        else pipe_trained.named_steps["reg"].coef_
    )
    # numerical feats (RBF + non-RBF feats) are not normalized
    coefs = coefs * X_test_pre.std(axis=0)
    # # numerical feats (RBF + non-RBF feats) are normalized
    # coefs
    df_coefs_scaled = (
        pd.DataFrame(coefs, columns=["Coefficient Importance"], index=trans_feat_names)
        .reset_index()
        .rename(columns={"index": "Feature"})
        .assign(
            Feature=lambda df: df["Feature"]
            .str[::-1]
            .str.replace("_", "=", 1)
            .str[::-1]
            .str.replace("_", " ")
            .str.title()
        )
        .sort_values(by=["Coefficient Importance"], axis=0, ascending=True)
        .assign(country=country_name)
    )
    print(f"Retrieved Coefficients for {country_name}")
    return df_coefs_scaled


def train_predict_single_ts(df, pipe, train, yvar_name="load"):
    # get name of timeseries (country)
    country_name = df.index.get_level_values(0).tolist()[0]
    # separate features from target
    X = df.drop(columns=[yvar_name])
    y = df[yvar_name]
    # get training split
    X_train = X.loc[train, :]
    y_train = y.to_frame().loc[train, :].squeeze()
    # train pipeline using training split
    _ = pipe.fit(X_train, y_train)
    # predict over full timeseries (train + test)
    y_pred = pd.Series(pipe.predict(X), index=X.index, name="pred")
    # get ML model coefficients from trained pipeline
    num_transformed_feats = pipe.named_steps["reg"].regressor_.coef_
    print(
        f"Country = {country_name}: Number of features = "
        f"{len(num_transformed_feats):,}"
    )
    return [y_pred, pipe]


def rmspe_error(y_true, y_pred):
    rmspe_val = np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true)), axis=0))
    return rmspe_val * 100


def smape_error(y_true, y_pred):
    n = len(y_pred)
    num = np.abs(y_pred - y_true)
    denom = np.abs(y_pred) + np.abs(y_true)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def calculate_evaluation_metrics(df):
    df_nontidy = df.pivot(
        index="utc_timestamp", columns=["variable"], values=["value"]
    ).reset_index()
    df_nontidy.columns = [
        "_".join(cp) for cp in df_nontidy.columns.to_flat_index().tolist()
    ]
    y_true = df_nontidy.set_index("utc_timestamp_")["value_load"]
    y_pred = df_nontidy.set_index("utc_timestamp_")["value_pred"]
    rmse = skm.mean_squared_error(y_true, y_pred, squared=False)
    r2 = skm.r2_score(y_true, y_pred)
    mae = skm.mean_absolute_error(y_true, y_pred)
    mse = skm.mean_squared_error(y_true, y_pred)
    rmspe = rmspe_error(y_true, y_pred)
    smape = smape_error(y_true, y_pred)
    df_metrics = pd.DataFrame.from_records(
        [
            {"metric": "rmse", "value": rmse},
            {"metric": "mae", "value": mae},
            {"metric": "smape(%)", "value": smape},
            {"metric": "mse", "value": mse},
            {"metric": "r2", "value": r2},
            {"metric": "rmspe(%)", "value": rmspe},
        ]
    )
    return df_metrics


def compare_forecasts(
    df,
    col_name="model_type",
    col_to_compare="value",
    search_strs=["ML Trained", "Naive"],
):
    fcast_1, fcast_2 = [
        df.query(f"{col_name}.str.startswith('{search_str}')")[
            col_to_compare
        ].reset_index(drop=True)
        for search_str in search_strs
    ]
    return (fcast_1 - fcast_2).mean()

In [7]:
def customize_splines(ax: plt.axis) -> plt.axis:
    ax.spines["left"].set_edgecolor("black")
    ax.spines["left"].set_linewidth(2)
    ax.spines["bottom"].set_edgecolor("black")
    ax.spines["bottom"].set_linewidth(2)
    ax.spines["top"].set_edgecolor("lightgrey")
    ax.spines["top"].set_linewidth(1)
    ax.spines["right"].set_edgecolor("lightgrey")
    ax.spines["right"].set_linewidth(1)
    return ax


def plot_bar_chart(
    data,
    xvar="metric:N",
    yvar="diff_pct_points:Q",
    ptitle="Percentage Points Difference between ML and Naive Forecasts",
    tooltip=[],
    fig_size=dict(width=450, height=200),
):
    chart = (
        alt.Chart(data, title=ptitle)
        .mark_bar(color="teal")
        .encode(
            x=alt.X(xvar, title=None, axis=alt.Axis(labelAngle=0)),
            y=alt.Y(yvar, title=None),
            tooltip=tooltip,
        )
        .properties(**fig_size)
    )
    rules = (
        alt.Chart(df_metrics_ppt_diff.assign(zero=0))
        .mark_rule(color="black")
        .encode(y=alt.Y("zero:Q"))
    )
    return chart + rules


def plot_multi_model_evaluation_metrics(
    df,
    ts_separator_values,
    tooltip,
    ts_separator_name="country",
    ptitle="Evaluation Metrics",
    ptitle_fontsize=14,
    ticklabel_fontsize=12,
    fig_size=dict(width=60, height=200),
):
    charts = []
    for k, c in enumerate(sorted(ts_separator_values)):
        if k > 0:
            alt_color = alt.Color("model_type:N", title=ptitle, legend=None)
        else:
            alt_color = alt.Color("model_type:N", title=ptitle)
        chart = (
            alt.Chart(df.query(f"{ts_separator_name} == '{c.split('_')[0]}'"))
            .mark_bar()
            .encode(
                x=alt.X(
                    "model_type:N",
                    title=None,
                    axis=alt.Axis(labels=False, domain=False, ticks=False),
                ),
                y=alt.Y("value:Q", title=None, axis=alt.Axis(grid=False, ticks=False)),
                color=alt_color,
                column=alt.Column(
                    "metric:N",
                    title=c,
                    spacing=15,
                    header=alt.Header(
                        titleOrient="top",
                        titlePadding=-5,
                        titleFontSize=ptitle_fontsize,
                        titleAnchor="start",
                        labelOrient="bottom",
                        labelPadding=fig_size["height"] + 10,
                        labelFontSize=ticklabel_fontsize,
                    ),
                ),
                tooltip=tooltip,
            )
            .properties(**fig_size)
            .configure_view(strokeOpacity=0)
            .resolve_scale(y="independent")
        )
        charts.append(chart)
    return charts


def plot_coef_bar_chart(
    data,
    xvar="Coefficient Importance",
    yvar="Feature:N",
    ptitle="ML Model Feature Coefficients",
    ptitle_x_loc=135,
    tick_label_fontsize=14,
    title_fontsize=16,
    color_by_col="gt_zero:N",
    tooltip=["Feature", "Coefficient Importance"],
    fig_size=dict(width=275, height=1200),
):
    country_name = data["country"].iloc[0]
    # print(country_name)
    # display(data)
    chart = (
        alt.Chart(data, title=f"{ptitle} in {country_name}")
        .mark_bar()
        .encode(
            x=alt.X(xvar, title=None),
            y=alt.Y(
                yvar,
                sort=None,
                title=None,
                axis=alt.Axis(labelFontSize=14, ticks=False),
            ),
            color=alt.Color(
                color_by_col,
                title=None,
                legend=None,
                scale=alt.Scale(domain=[False, True], range=["red", "green"]),
            ),
            tooltip=tooltip,
        )
        .properties(**fig_size)
    )
    rules = (
        alt.Chart(df_coefs_scaled.assign(vline=0))
        .mark_rule(color="black")
        .encode(x="vline:Q")
    )
    combo = (chart + rules).configure_title(
        fontSize=title_fontsize, anchor="start", align="left", dx=ptitle_x_loc
    )
    return combo


def plot_y_vs_x(
    data,
    xvar,
    yvar,
    xvar_axis_label,
    yvar_axis_label,
    ptitle,
    ax,
    axis_label_fontsize=12,
    tick_label_fontsize=12,
    ptitle_fontsize=12,
    plot_hline=False,
    diag_line_coords=[],
):
    data.plot(
        ax=ax,
        x=xvar,
        y=yvar,
        kind="scatter",
        edgecolor="blue",
        zorder=3,
        s=40,
        c="none",
    )
    ax.set_xlabel(xvar_axis_label, fontsize=axis_label_fontsize)
    ax.set_ylabel(yvar_axis_label, fontsize=axis_label_fontsize)
    ax.xaxis.set_tick_params(labelsize=tick_label_fontsize)
    ax.yaxis.set_tick_params(labelsize=tick_label_fontsize)
    ax.grid(color="lightgrey", zorder=0)
    if plot_hline:
        ax.axhline(y=0, lw=2, c="red", ls="--", zorder=3)
    if diag_line_coords:
        data = data.dropna()
        m, b = np.polyfit(data[xvar].to_numpy(), data[yvar].to_numpy(), 1)
        ax.plot(
            data[xvar].to_numpy(),
            m * data[xvar].to_numpy() + b,
            color="black",
            lw=1.5,
            zorder=3,
        )
        r2 = skm.r2_score(data[xvar].to_numpy(), data[yvar].to_numpy())
        lows, highs = diag_line_coords[0], diag_line_coords[1]
        ax.axline(lows, highs, c="red", zorder=3, ls="--")
        ptitle += r" ($\mathregular{R^2}$" + f"={r2:.2f})"
    ax.set_title(ptitle, fontsize=ptitle_fontsize, fontweight="bold", loc="left")
    _ = customize_splines(ax)


def plot_true_pred(df, ylabel, fig_properties=dict(width=1600, height=350)):
    # display(df.head())
    chart = (
        alt.Chart(df)
        .mark_line(opacity=0.6)
        .encode(
            x=alt.X("utc_timestamp:T", title=None),
            y=alt.Y(
                "value:Q",
                title=ylabel,
                scale=alt.Scale(domain=[df["value"].min(), df["value"].max()]),
            ),
            color=alt.Color(
                "variable:N",
                title=None,
                scale=alt.Scale(
                    domain=["load", "pred"],
                    range=["green", "red"],
                ),
            ),
            tooltip=["split_type", "variable", "value"],
        )
    )
    rect = (
        alt.Chart(df)
        .mark_rect(
            fill="#e6f5c9",
            stroke=None,
            fillOpacity=0.1,
            opacity=0.4,
        )
        .encode(
            x=alt.X("test_starts:T"),
            x2="test_ends:T",
            y=alt.value(0),
            y2=alt.value(fig_properties["height"]),
            # color="color:N",
            tooltip=["split_type"],
        )
    )
    combo = (
        alt.layer(rect, chart)
        .properties(**fig_properties)
        .configure_title(
            fontSize=14,
            anchor="start",
            align="left",
            # dx=40,
        )
    )
    return combo


def plot_histogram(data, ptitle, figsize=dict(width=375, height=300)):
    hist_chart = (
        alt.Chart()
        .mark_bar(binSpacing=0, color="#1f77b4")
        .encode(
            x=alt.X("residual:Q", bin=alt.Bin(maxbins=100)),
            y=alt.Y("count()", title=None, stack=None, scale=alt.Scale(type="linear")),
        )
        .properties(**figsize)
    )
    rules = (
        alt.Chart()
        .mark_rule(color="red", strokeWidth=1.5)
        .encode(x=alt.X("zero:Q"), tooltip=["zero"])
    )
    combo_hist = (
        alt.layer(
            hist_chart,
            rules,
            data=data,
        )
        .facet(
            column=alt.Column(
                "split:N",
                title=ptitle,
                sort=["train", "test"],
                header=alt.Header(
                    titleOrient="top",
                    titleAlign="left",
                    titleAnchor="start",
                    labelAnchor="end",
                    labelOrient="bottom",
                    titlePadding=-5,
                    labelFontSize=12,
                    titleFontSize=14,
                ),
            )
        )
        .resolve_scale(y="independent")
    )
    return combo_hist


def plot_diagnostic_charts(
    df_pred: pd.DataFrame,
    c: str,
    ptitle_hist: str = "Fit Residual",
    yvar: str = "Load",
    ptitle_true_pred_prefix: str = "True vs Predicted Load in",
    tick_label_fontsize: int = 12,
    ptitle_fontsize: int = 14,
    ptitle_qq: str = "Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x: str = "Predicted vs Observed Test Split",
    ptitle_acf: str = "ACF for Fit Residual of Test Split",
    n_lags_acf: int = 56,
) -> None:
    # true vs predicted
    data_country = df_pred.query(f"country == '{c}'")
    true_pred_chart = plot_true_pred(
        data_country,
        yvar,
        fig_properties=dict(
            width=800,
            height=300,
            title=f"{ptitle_true_pred_prefix} {c}",
        ),
    )

    # histogram of test split residual
    y_train_true = data_country.query(
        "(split_type == 'train') & (variable == 'load')"
    ).set_index("utc_timestamp")["value"]
    y_train_pred = data_country.query(
        "(split_type == 'train') & (variable == 'pred')"
    ).set_index("utc_timestamp")["value"]
    y_test_true = data_country.query(
        "(split_type == 'test') & (variable == 'load')"
    ).set_index("utc_timestamp")["value"]
    y_test_pred = data_country.query(
        "(split_type == 'test') & (variable == 'pred')"
    ).set_index("utc_timestamp")["value"]
    data_chart_hist = pd.concat(
        [
            (y_train_true - y_train_pred)
            .rename("residual")
            .to_frame()
            .assign(country=c)
            .assign(split="train"),
            (y_test_true - y_test_pred)
            .rename("residual")
            .to_frame()
            .assign(country=c)
            .assign(split="test"),
        ]
    )
    combo_hist = plot_histogram(
        data_chart_hist.assign(zero=0),
        ptitle_hist,
        figsize=dict(width=375, height=300),
    )

    display(true_pred_chart)
    display(combo_hist)

    fig = plt.figure(figsize=(14, 6))
    grid = plt.GridSpec(1, 2, hspace=0.2, wspace=0.2)
    ax1 = fig.add_subplot(grid[0, 0], xticklabels=[])
    ax2 = fig.add_subplot(grid[0, 1], xticklabels=[])

    # QQ plot for test split residual
    ts = (y_test_true - y_test_pred).rename("residual")
    _ = sm.qqplot(ts.dropna(how="any"), fit=True, line="45", ax=ax1)
    ax1.set_title(ptitle_qq, loc="left", fontweight="bold", fontsize=ptitle_fontsize)
    ax1.set_xlabel(ax1.get_xlabel(), fontsize=tick_label_fontsize)
    ax1.set_ylabel(ax1.get_ylabel(), fontsize=tick_label_fontsize)
    ax1.grid(which="both", axis="both", color="lightgrey")
    _ = customize_splines(ax1)

    # true vs predicted for test split
    data_xy = (
        data_country.query("split_type == 'test'")
        .pivot(index=["utc_timestamp"], columns=["variable"], values=["value"])
        .reset_index()
    )
    data_xy.columns = ["_".join(cp) for cp in data_xy.columns.to_flat_index().tolist()]
    lows_highs = [
        [data_xy[["value_load", "value_pred"]].min().min()] * 2,
        [data_xy[["value_load", "value_pred"]].max().max()] * 2,
    ]
    plot_y_vs_x(
        data_xy,
        "value_load",
        "value_pred",
        "Observed",
        "Predicted",
        ptitle_y_vs_x,
        ax2,
        axis_label_fontsize=tick_label_fontsize,
        tick_label_fontsize=tick_label_fontsize,
        ptitle_fontsize=ptitle_fontsize,
        plot_hline=False,
        diag_line_coords=lows_highs,
    )

    # Auto-Correlation plot for test split residual
    y_test_res = (y_test_true - y_test_pred).rename("residual")
    a, b, c, d = acf(y_test_res, alpha=0.05, nlags=n_lags_acf, fft=True, qstat=True)
    df_acf = pd.DataFrame.from_dict({"ACF": a, "q_stat": c, "p": d}, orient="index").T
    df_acf = (
        df_acf.merge(
            pd.DataFrame(b, columns=["low", "high"]), left_index=True, right_index=True
        )
        .reset_index()
        .rename(columns={"index": "lag"})
    )
    df_acf["lag_min"] = 0
    df_acf["lag_max"] = n_lags_acf
    points = (
        alt.Chart(df_acf, title=ptitle_acf)
        .mark_point(fill="blue", size=70, strokeWidth=1.5, color="black", opacity=1)
        .encode(
            y=alt.Y("ACF", axis=alt.Axis(title="")),
            x=alt.X("lag", axis=alt.Axis(domainWidth=1.5, domainColor="black")),
        )
    )
    rules = alt.Chart(df_acf).mark_rule(size=2).encode(x="lag", y="ACF")
    combo_acf_chart = (
        alt.layer(rules, points)
        .configure_title(anchor="start", offset=-3, fontSize=ptitle_fontsize)
        .configure_axis(
            labelFontSize=tick_label_fontsize, titleFontSize=tick_label_fontsize
        )
    )
    display(combo_acf_chart)

## Get Data and Feature Engineering

We will
- load raw electricity consumption data for the selected countries from the [OPSD portal](https://data.open-power-system-data.org/time_series/)
- add the following `datetime` features
  - season
  - month
  - week of year
  - week of month
  - hour of day
  - check if day is a weekday (Mon - Fri)
  - check if day is a holiday

  to all selected countries timeseries

In [8]:
%%time
df = (
    pd.read_csv(
        opsd_data_url,
        # nrows=500,
        usecols=raw_mask,
        parse_dates=["utc_timestamp"],
        index_col="utc_timestamp",
    )
    .loc[slice(str(start_date), str(end_date))]
    .stack()
    .reset_index()
    .assign(country=lambda df: df["level_1"].str.split("_", expand=True)[0])
    .drop(columns=["level_1"])
    .rename(columns={0: "load"})
    .assign(load=lambda df: df["load"] / 1000)
    .assign(season=lambda df: df["utc_timestamp"].dt.month.astype(str).map(seasons))
    .assign(month=lambda df: df["utc_timestamp"].dt.month_name())
    .assign(week_of_year=lambda df: df["utc_timestamp"].dt.isocalendar().week)
    .assign(wom=lambda df: (df["utc_timestamp"].dt.day - 1) // 7 + 1)
    # .assign(day_of_year=lambda df: df['utc_timestamp'].dt.day)
    .assign(hour_of_day=lambda df: df["utc_timestamp"].dt.hour)
    .assign(is_weekday=lambda df: df["utc_timestamp"].dt.weekday.isin([0, 1, 2, 3, 4]))
    # .assign(period_num=lambda df: np.arange(len(df.index)))
)
dfs = []
for c, h in zip(country_names, holidays_objs):
    df_country = df.query(f"country == '{c.split('_')[0]}'").copy()
    df_country["is_holiday"] = df_country["utc_timestamp"].dt.date.apply(
        lambda x: x in h
    )
    dfs.append(df_country)
df = pd.concat(dfs, ignore_index=True).set_index(['country', 'utc_timestamp']).sort_index()
display(df.head())
display(df.tail())

load  season    month  week_of_year  wom  \
country utc_timestamp                                                          
AT      2015-01-01 00:00:00+00:00  5.946  winter  January             1    1   
        2015-01-01 01:00:00+00:00  5.726  winter  January             1    1   
        2015-01-01 02:00:00+00:00  5.347  winter  January             1    1   
        2015-01-01 03:00:00+00:00  5.249  winter  January             1    1   
        2015-01-01 04:00:00+00:00  5.309  winter  January             1    1   

                                   hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                   
AT      2015-01-01 00:00:00+00:00            0        True        True  
        2015-01-01 01:00:00+00:00            1        True        True  
        2015-01-01 02:00:00+00:00            2        True        True  
        2015-01-01 03:00:00+00:00            3        True        True  
        2015-01-01 04:00:00+00:00            4        True        True

load  season      month  week_of_year  \
country utc_timestamp                                                       
SK      2020-09-30 18:00:00+00:00  3.584  summer  September            40   
        2020-09-30 19:00:00+00:00  3.324  summer  September            40   
        2020-09-30 20:00:00+00:00  3.099  summer  September            40   
        2020-09-30 21:00:00+00:00  2.940  summer  September            40   
        2020-09-30 22:00:00+00:00  2.823  summer  September            40   

                                   wom  hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                        
SK      2020-09-30 18:00:00+00:00    5           18        True       False  
        2020-09-30 19:00:00+00:00    5           19        True       False  
        2020-09-30 20:00:00+00:00    5           20        True       False  
        2020-09-30 21:00:00+00:00    5           21        True       False  
        2020-09-30 22:00:00+00:00    5           22        True       False

CPU times: user 4.42 s, sys: 748 ms, total: 5.17 s
Wall time: 16.1 s


## Train-Test Split

Select the range of dates wanted from the data

In [9]:
data = df.loc[full, :]
data

load  season      month  week_of_year  \
country utc_timestamp                                                       
AT      2018-01-01 00:00:00+00:00  5.934  winter    January             1   
        2018-01-01 01:00:00+00:00  5.729  winter    January             1   
        2018-01-01 02:00:00+00:00  5.467  winter    January             1   
        2018-01-01 03:00:00+00:00  5.327  winter    January             1   
        2018-01-01 04:00:00+00:00  5.423  winter    January             1   
...                                  ...     ...        ...           ...   
SK      2019-09-30 19:00:00+00:00  3.282  summer  September            40   
        2019-09-30 20:00:00+00:00  3.081  summer  September            40   
        2019-09-30 21:00:00+00:00  2.934  summer  September            40   
        2019-09-30 22:00:00+00:00  2.816  summer  September            40   
        2019-09-30 23:00:00+00:00  2.727  summer  September            40   

                                   wom  hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                        
AT      2018-01-01 00:00:00+00:00    1            0        True        True  
        2018-01-01 01:00:00+00:00    1            1        True        True  
        2018-01-01 02:00:00+00:00    1            2        True        True  
        2018-01-01 03:00:00+00:00    1            3        True        True  
        2018-01-01 04:00:00+00:00    1            4        True        True  
...                                ...          ...         ...         ...  
SK      2019-09-30 19:00:00+00:00    5           19        True       False  
        2019-09-30 20:00:00+00:00    5           20        True       False  
        2019-09-30 21:00:00+00:00    5           21        True       False  
        2019-09-30 22:00:00+00:00    5           22        True       False  
        2019-09-30 23:00:00+00:00    5           23        True       False  

[183733 rows x 8 columns]

Divide the data into
- training
  - 2018-01-01 to 2019-08-31
- testing
  - 2019-09-01 to 2019-09-30

splits

In [10]:
data_train = data.loc[train, :]
data_test = data.loc[test, :]
print(
    f"Training data contains {len(data_train):,} rows\nTesting data contains {len(data_test):,} rows"
)
display(data_train.head(3))
display(data_train.tail(3))
display(data_test.head(3))
display(data_test.tail(3))

Training data contains 175,093 rows
Testing data contains 8,640 rows


load  season    month  week_of_year  wom  \
country utc_timestamp                                                          
AT      2018-01-01 00:00:00+00:00  5.934  winter  January             1    1   
        2018-01-01 01:00:00+00:00  5.729  winter  January             1    1   
        2018-01-01 02:00:00+00:00  5.467  winter  January             1    1   

                                   hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                   
AT      2018-01-01 00:00:00+00:00            0        True        True  
        2018-01-01 01:00:00+00:00            1        True        True  
        2018-01-01 02:00:00+00:00            2        True        True

load  season   month  week_of_year  wom  \
country utc_timestamp                                                         
SK      2019-08-31 21:00:00+00:00  2.663  summer  August            35    5   
        2019-08-31 22:00:00+00:00  2.567  summer  August            35    5   
        2019-08-31 23:00:00+00:00  2.455  summer  August            35    5   

                                   hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                   
SK      2019-08-31 21:00:00+00:00           21       False       False  
        2019-08-31 22:00:00+00:00           22       False       False  
        2019-08-31 23:00:00+00:00           23       False       False

load  season      month  week_of_year  \
country utc_timestamp                                                       
AT      2019-09-01 00:00:00+00:00  4.726  summer  September            35   
        2019-09-01 01:00:00+00:00  4.592  summer  September            35   
        2019-09-01 02:00:00+00:00  4.513  summer  September            35   

                                   wom  hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                        
AT      2019-09-01 00:00:00+00:00    1            0       False       False  
        2019-09-01 01:00:00+00:00    1            1       False       False  
        2019-09-01 02:00:00+00:00    1            2       False       False

load  season      month  week_of_year  \
country utc_timestamp                                                       
SK      2019-09-30 21:00:00+00:00  2.934  summer  September            40   
        2019-09-30 22:00:00+00:00  2.816  summer  September            40   
        2019-09-30 23:00:00+00:00  2.727  summer  September            40   

                                   wom  hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                        
SK      2019-09-30 21:00:00+00:00    5           21        True       False  
        2019-09-30 22:00:00+00:00    5           22        True       False  
        2019-09-30 23:00:00+00:00    5           23        True       False

**Notes**
1. The features created in the previous section do not leak any information from the training split into the testing split and so it was valid to engineer those features before the data was split.

## Separate Features from Target

Extract the
- independent (ML features)
- dependent target (electricity consumption, or `load`)

variables (columns) from the data 

In [11]:
X_train = data_train.drop(columns=["load"]).copy()
X_test = data_test.drop(columns=["load"]).copy()
X = pd.concat([X_train, X_test])
y_train = data_train["load"].copy()
y_test = data_test["load"].copy()
display(X)
display(X_train)
display(X_test)
display(y_train.to_frame())
display(y_test.to_frame())

season      month  week_of_year  wom  \
country utc_timestamp                                                     
AT      2018-01-01 00:00:00+00:00  winter    January             1    1   
        2018-01-01 01:00:00+00:00  winter    January             1    1   
        2018-01-01 02:00:00+00:00  winter    January             1    1   
        2018-01-01 03:00:00+00:00  winter    January             1    1   
        2018-01-01 04:00:00+00:00  winter    January             1    1   
...                                   ...        ...           ...  ...   
SK      2019-09-30 19:00:00+00:00  summer  September            40    5   
        2019-09-30 20:00:00+00:00  summer  September            40    5   
        2019-09-30 21:00:00+00:00  summer  September            40    5   
        2019-09-30 22:00:00+00:00  summer  September            40    5   
        2019-09-30 23:00:00+00:00  summer  September            40    5   

                                   hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                   
AT      2018-01-01 00:00:00+00:00            0        True        True  
        2018-01-01 01:00:00+00:00            1        True        True  
        2018-01-01 02:00:00+00:00            2        True        True  
        2018-01-01 03:00:00+00:00            3        True        True  
        2018-01-01 04:00:00+00:00            4        True        True  
...                                        ...         ...         ...  
SK      2019-09-30 19:00:00+00:00           19        True       False  
        2019-09-30 20:00:00+00:00           20        True       False  
        2019-09-30 21:00:00+00:00           21        True       False  
        2019-09-30 22:00:00+00:00           22        True       False  
        2019-09-30 23:00:00+00:00           23        True       False  

[183733 rows x 7 columns]

season    month  week_of_year  wom  \
country utc_timestamp                                                   
AT      2018-01-01 00:00:00+00:00  winter  January             1    1   
        2018-01-01 01:00:00+00:00  winter  January             1    1   
        2018-01-01 02:00:00+00:00  winter  January             1    1   
        2018-01-01 03:00:00+00:00  winter  January             1    1   
        2018-01-01 04:00:00+00:00  winter  January             1    1   
...                                   ...      ...           ...  ...   
SK      2019-08-31 19:00:00+00:00  summer   August            35    5   
        2019-08-31 20:00:00+00:00  summer   August            35    5   
        2019-08-31 21:00:00+00:00  summer   August            35    5   
        2019-08-31 22:00:00+00:00  summer   August            35    5   
        2019-08-31 23:00:00+00:00  summer   August            35    5   

                                   hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                   
AT      2018-01-01 00:00:00+00:00            0        True        True  
        2018-01-01 01:00:00+00:00            1        True        True  
        2018-01-01 02:00:00+00:00            2        True        True  
        2018-01-01 03:00:00+00:00            3        True        True  
        2018-01-01 04:00:00+00:00            4        True        True  
...                                        ...         ...         ...  
SK      2019-08-31 19:00:00+00:00           19       False       False  
        2019-08-31 20:00:00+00:00           20       False       False  
        2019-08-31 21:00:00+00:00           21       False       False  
        2019-08-31 22:00:00+00:00           22       False       False  
        2019-08-31 23:00:00+00:00           23       False       False  

[175093 rows x 7 columns]

season      month  week_of_year  wom  \
country utc_timestamp                                                     
AT      2019-09-01 00:00:00+00:00  summer  September            35    1   
        2019-09-01 01:00:00+00:00  summer  September            35    1   
        2019-09-01 02:00:00+00:00  summer  September            35    1   
        2019-09-01 03:00:00+00:00  summer  September            35    1   
        2019-09-01 04:00:00+00:00  summer  September            35    1   
...                                   ...        ...           ...  ...   
SK      2019-09-30 19:00:00+00:00  summer  September            40    5   
        2019-09-30 20:00:00+00:00  summer  September            40    5   
        2019-09-30 21:00:00+00:00  summer  September            40    5   
        2019-09-30 22:00:00+00:00  summer  September            40    5   
        2019-09-30 23:00:00+00:00  summer  September            40    5   

                                   hour_of_day  is_weekday  is_holiday  
country utc_timestamp                                                   
AT      2019-09-01 00:00:00+00:00            0       False       False  
        2019-09-01 01:00:00+00:00            1       False       False  
        2019-09-01 02:00:00+00:00            2       False       False  
        2019-09-01 03:00:00+00:00            3       False       False  
        2019-09-01 04:00:00+00:00            4       False       False  
...                                        ...         ...         ...  
SK      2019-09-30 19:00:00+00:00           19        True       False  
        2019-09-30 20:00:00+00:00           20        True       False  
        2019-09-30 21:00:00+00:00           21        True       False  
        2019-09-30 22:00:00+00:00           22        True       False  
        2019-09-30 23:00:00+00:00           23        True       False  

[8640 rows x 7 columns]

load
country utc_timestamp                   
AT      2018-01-01 00:00:00+00:00  5.934
        2018-01-01 01:00:00+00:00  5.729
        2018-01-01 02:00:00+00:00  5.467
        2018-01-01 03:00:00+00:00  5.327
        2018-01-01 04:00:00+00:00  5.423
...                                  ...
SK      2019-08-31 19:00:00+00:00  2.942
        2019-08-31 20:00:00+00:00  2.790
        2019-08-31 21:00:00+00:00  2.663
        2019-08-31 22:00:00+00:00  2.567
        2019-08-31 23:00:00+00:00  2.455

[175093 rows x 1 columns]

load
country utc_timestamp                   
AT      2019-09-01 00:00:00+00:00  4.726
        2019-09-01 01:00:00+00:00  4.592
        2019-09-01 02:00:00+00:00  4.513
        2019-09-01 03:00:00+00:00  4.639
        2019-09-01 04:00:00+00:00  4.837
...                                  ...
SK      2019-09-30 19:00:00+00:00  3.282
        2019-09-30 20:00:00+00:00  3.081
        2019-09-30 21:00:00+00:00  2.934
        2019-09-30 22:00:00+00:00  2.816
        2019-09-30 23:00:00+00:00  2.727

[8640 rows x 1 columns]

Count the number of days in the training data, ending with the day before the first day of the test split (start of the pilot study)

In [12]:
num_days_train = (
    X_train.loc[(country_names[0], slice(None))].index.get_level_values(0).max()
    + pd.Timedelta(1, unit="h")
    - X_train.loc[(country_names[0], slice(None))].index.get_level_values(0).min()
).days
print(
    f"The training data covers the {num_days_train} most recent days of "
    "electricity consumption data available per country."
)

The training data covers the 608 most recent days of electricity consumption data available per country.


## Exploratory Data Analysis

There are no numerical variables in the data since all the engineered datetime attributes are categoricals. So, we don't have features that are correlated to eachother. We've avoided the [multi-collinearity problem for linear models](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html#id6), which will be used to generate the forecast.

## Target Processing

Fill missing values in test split of the target (electricity consumption column, `load`) with average of values from training split

In [13]:
%%time
y_filled = data.groupby(["country"]).apply(
    fill_target_nans, train=train, test=test, y_col_name="load"
)
y_train = y_filled.loc[train, :].squeeze().copy()
y_test = y_filled.loc[test, :].squeeze().copy()
y = y_filled.squeeze().copy()

CPU times: user 202 ms, sys: 59 µs, total: 202 ms
Wall time: 201 ms


**Notes**
1. This filling to missing values has to be done per timeseries (country) and so we apply a helper function `fill_target_nans()` inside a `.groupby()` in order to do this separately for each timeseries.

## Feature Transformation and ML Model Pipeline

(OPTION 1) Generate RBF features for
- `hour of day`

and categoricals for
- `week_of_year`
- `is_weekday`
- `is_holiday`

In [15]:
rbf_hourly_feats = ["hour_of_day"]
# NUMS = ["period_num"]  # ("nums-passthrough", PassthroughTransformer(), NUMS)
# NUMS = ["week_of_year"]  # ("nums-passthrough", StandardScaler(), NUMS)
NUMS = []
CATS = [
    "is_weekday",
    "is_holiday",
    # "month",
    "week_of_year",
    # "season",
    # "wom",
]

rbf_hour_of_day = RepeatingBasisFunction(
    n_periods=7, remainder="passthrough", column="hour_of_day", input_range=(1, 24)
)
preprocessor = ColumnTransformer(
    [
        ("nums-rbf-hour", rbf_hour_of_day, rbf_hourly_feats),
        ("cats", OneHotEncoder(drop="first", dtype="int64"), CATS),
    ],
    remainder="drop",
)

(OPTION 2) Generate RBF features for
- `hour of day`
- `week_of_year`

and categoricals for
- `is_weekday`
- `is_holiday`

Here, option 1 will be used.

In [16]:
# rbf_hourly_feats = ["hour_of_day"]
# rbf_weekly_feats = ["week_of_year"]
# # NUMS = ["period_num"]  # ("nums-passthrough", PassthroughTransformer(), NUMS)
# CATS = [
#     "is_weekday",
#     "is_holiday",
#     # "month",
#     # "season",
#     # "wom",
# ]

# rbf_week_of_year = RepeatingBasisFunction(
#     n_periods=10, remainder="passthrough", column="week_of_year", input_range=(1, 53)
# )
# rbf_hour_of_day = RepeatingBasisFunction(
#     n_periods=7, remainder="passthrough", column="hour_of_day", input_range=(1, 24)
# )
# preprocessor = ColumnTransformer(
#     [
#         ("nums-rbf-hour", rbf_hour_of_day, rbf_hourly_feats),
#         ("nums-rbf-week", rbf_week_of_year, rbf_weekly_feats),
#         ("cats", OneHotEncoder(drop="first", dtype="int64"), CATS),
#     ],
#     remainder="drop",
# )

Next, we will combine the preprocessing and ML modeling steps into a single pipeline, where the target (`load` column) is transformed using a natural logarithm (log base-10) before training the model and the resulting predictions will be reverted using an exponential transform ([link](https://optimization.cbe.cornell.edu/index.php?title=Exponential_transformation))

In [17]:
# pipe = Pipeline([("pre", preprocessor), ("reg", LinearRegression())])
pipe = Pipeline(
    [
        ("pre", preprocessor),
        (
            "reg",
            TransformedTargetRegressor(
                regressor=LinearRegression(),
                func=np.log10,
                inverse_func=sp.special.exp10,
            ),
        ),
    ]
)

## ML Model (Forecast) Training

We'll now use the above pipeline to
- encode the data using
  - RBF features for
    - hour of the day
  - one hot encoding for
    - week of year
    - month
    - check if day is a holiday
    - check if day is a weekday (Mon-Fri)
- train a linear regression model with all engineered and (manually) selected features

In [18]:
%%time
# # using GROUP BY (does not work)
# y_preds_pipelines = (
#     pd.concat([X, y], axis=1).sort_values(by=["country", "utc_timestamp"])
#     .groupby(["country"])
#     .apply(train_predict_single_ts, pipe=pipe, train=train, yvar_name="load")
# )
# y_pred = pd.concat([ypp[0] for ypp in y_preds_pipelines])
# pipes_trained = {c.split('_')[0]:ypp[1] for ypp, c in zip(y_preds_pipelines, country_names)}

# using for loop
y_preds_countries = []
dfs_coefs_scaled = []
for c in country_names:
    y_pred_sc, pipe_sc = train_predict_single_ts(
        pd.concat([X, y], axis=1)
        .query(f"country == '{c.split('_')[0]}'")
        .sort_values(by=["country", "utc_timestamp"]),
        pipe=pipe,
        train=train,
        yvar_name="load",
    )
    df_single_ts_coefs_scaled = get_coefs_feats(
        pipe_sc, rbf_hourly_feats, NUMS, CATS, X_test, c
    )
    y_preds_countries.append(y_pred_sc)
    dfs_coefs_scaled.append(df_single_ts_coefs_scaled)
y_pred = pd.concat(y_preds_countries)
df_coefs_scaled = pd.concat(dfs_coefs_scaled, ignore_index=True)
df_coefs_untidy = (
    df_coefs_scaled.pivot(
        index="Feature", columns=["country"], values=["Coefficient Importance"]
    )
    .T
    .reset_index(level=1)
    .reset_index(drop=True)
    .set_index('country')
    .astype(pd.Float32Dtype())
    .reset_index()
)
with pd.option_context('display.max_columns', None):
    display(df_coefs_untidy)

Country = DE: Number of features = 60
Retrieved Coefficients for DE
Country = CZ: Number of features = 60
Retrieved Coefficients for CZ
Country = BE: Number of features = 60
Retrieved Coefficients for BE
Country = HR: Number of features = 60
Retrieved Coefficients for HR
Country = PL: Number of features = 60
Retrieved Coefficients for PL
Country = IT: Number of features = 60
Retrieved Coefficients for IT
Country = SK: Number of features = 60
Retrieved Coefficients for SK
Country = RO: Number of features = 60
Retrieved Coefficients for RO
Country = ES: Number of features = 60
Retrieved Coefficients for ES
Country = AT: Number of features = 60
Retrieved Coefficients for AT
Country = DK: Number of features = 60
Retrieved Coefficients for DK
Country = GB: Number of features = 60
Retrieved Coefficients for GB_GBN


Feature,country,Is Holiday=True,Is Weekday=True,Rbf=1,Rbf=2,Rbf=3,Rbf=4,Rbf=5,Rbf=6,Rbf=7,Week Of Year=10,Week Of Year=11,Week Of Year=12,Week Of Year=13,Week Of Year=14,Week Of Year=15,Week Of Year=16,Week Of Year=17,Week Of Year=18,Week Of Year=19,Week Of Year=2,Week Of Year=20,Week Of Year=21,Week Of Year=22,Week Of Year=23,Week Of Year=24,Week Of Year=25,Week Of Year=26,Week Of Year=27,Week Of Year=28,Week Of Year=29,Week Of Year=3,Week Of Year=30,Week Of Year=31,Week Of Year=32,Week Of Year=33,Week Of Year=34,Week Of Year=35,Week Of Year=36,Week Of Year=37,Week Of Year=38,Week Of Year=39,Week Of Year=4,Week Of Year=40,Week Of Year=41,Week Of Year=42,Week Of Year=43,Week Of Year=44,Week Of Year=45,Week Of Year=46,Week Of Year=47,Week Of Year=48,Week Of Year=49,Week Of Year=5,Week Of Year=50,Week Of Year=51,Week Of Year=52,Week Of Year=6,Week Of Year=7,Week Of Year=8,Week Of Year=9
0,AT,-0.006549,0.037532,-0.037588,-0.004738,0.021373,0.003102,0.007979,0.008829,-0.001262,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.009821,-0.024246,-0.021003,-0.019679,-0.019667,0.0,-0.007214,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,BE,-0.00507,0.025591,-0.023984,-0.00878,0.014847,0.00254,0.00422,0.005212,0.004607,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.005372,-0.011667,-0.012325,-0.013505,-0.010134,0.0,-0.004895,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,CZ,-0.005789,0.032148,1.847153,1.743832,1.777629,1.7726,1.770176,1.76281,1.784958,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.007567,-0.020857,-0.019445,-0.019322,-0.01471,0.0,-0.003077,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,DE,-0.006489,0.039189,-0.034444,-0.004457,0.014342,0.010904,0.001682,0.011596,-0.001708,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.001405,-0.002325,-0.001844,0.003071,0.001841,0.0,-0.00177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,DK,-0.003803,0.028115,-2.864934,-2.642111,-2.634609,-2.655938,-2.650735,-2.633764,-2.698691,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.009845,-0.02545,-0.023358,-0.022358,-0.016731,0.0,-0.005717,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
5,ES,-0.005658,0.028578,-2.302256,-2.124355,-2.121114,-2.118587,-2.13086,-2.118015,-2.14024,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.003654,0.005016,0.002062,0.007206,0.002747,0.0,-0.00178,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
6,GB_GBN,-0.002547,0.019425,-0.036967,-0.032654,0.021964,0.011424,0.007545,0.017983,0.008804,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.019216,-0.038918,-0.040451,-0.036894,-0.026795,0.0,-0.011698,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0
7,HR,-0.00474,0.02003,-0.055733,-0.006239,0.014919,0.012793,-0.004255,0.015485,0.019981,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000628,-0.012195,-0.010311,-0.012203,-0.019937,0.0,-0.008358,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
8,IT,-0.008436,0.044005,-0.045063,-0.016682,0.026329,0.000872,0.00777,0.017559,0.006601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.009784,0.020872,0.024875,0.024668,0.014401,0.0,0.004455,0.0,0.0,0.0,0.0

CPU times: user 24.6 s, sys: 12.2 s, total: 36.9 s
Wall time: 21.7 s


**Notes**
1. The approach of training a single model on multiple timeseries (using the data processed as above) using the following approach
   ```python
   # ALL timeseries at once - gives poor performance
   _ = pipe.fit(X_train, y_train)
   y_pred = pd.Series(pipe.predict(X), index=X.index, name="pred")
   num_transformed_feats = pipe.named_steps["reg"].coef_
   print(f"Number of features = {len(num_transformed_feats):,}")
   y_pred.sort_index().loc[test].loc["DE"].to_frame().assign(
       load=y_test.sort_index().loc[test].loc["DE"].to_frame()
   )
   ```

   likely gives poor performance since
   - the `is_holiday` feature is mixed across all countries and loses its predictive power
   - a feature to separate the timeseries (country) is not present in the data

We'll now arrange the true and ML-based predicted values of the target (electricity load) into a [tidy](https://www.jstatsoft.org/article/view/v059i10) format that is suitable for plotting

In [19]:
%%time
dfs_pred = []
for split, indexes in zip(["train", "test"], [train, test]):
    df_split_pred = (
        pd.concat([y.to_frame(), y_pred.to_frame()], axis=1).sort_index()
        .loc[indexes, :]
        .stack()
        .reset_index()
        .rename(columns={"level_2": "variable", 0: "value"})
        .assign(split_type=split)
        .assign(test_starts=data_test.index.get_level_values(1).min())
        .assign(test_ends=data_test.index.get_level_values(1).max())
    )
    dfs_pred.append(df_split_pred)
df_pred = pd.concat(dfs_pred, ignore_index=True)
df_pred

CPU times: user 4.96 s, sys: 30 ms, total: 4.99 s
Wall time: 4.99 s


,country,utc_timestamp,variable,value,split_type,test_starts,test_ends
0,AT,2018-01-01 00:00:00+00:00,load,5.934000,train,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
1,AT,2018-01-01 00:00:00+00:00,pred,5.507641,train,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
2,AT,2018-01-01 01:00:00+00:00,load,5.729000,train,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
3,AT,2018-01-01 01:00:00+00:00,pred,5.367385,train,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
4,AT,2018-01-01 02:00:00+00:00,load,5.467000,train,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
...,...,...,...,...,...,...,...
367461,SK,2019-09-30 21:00:00+00:00,pred,3.228006,test,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
367462,SK,2019-09-30 22:00:00+00:00,load,2.816000,test,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
367463,SK,2019-09-30 22:00:00+00:00,pred,3.053353,test,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00
367464,SK,2019-09-30 23:00:00+00:00,load,2.727000,test,2019-09-01 00:00:00+00:00,2019-09-30 23:00:00+00:00


## Naive Forecast

When working with timeseries data, it is important to assess the performance of basic or naive forecasting methods prior to developing more sophisticated methods. Often, such approaches can provide a reliable lower bound on forecast performance against which the performance of a sophisticated forecasting technique can be assessed.

The naive strategy we will use here is to take the the electricity usage during the same period as the holdout data but from previous year (2018). We will naively assume that this consumption is the forecasted load for the holdout period (2019).

The naive forecast is generated below

In [20]:
%%time
test_last_year = y_test.index.get_level_values(1) - pd.Timedelta(364, unit="days")
y_test_pred_naive = y.loc[(slice(None), test_last_year)].sort_index()
df_pred_naive = (
    pd.DataFrame()
    .assign(load=y_test)
    .assign(
        pred=pd.Series(
            y_test_pred_naive
            .reset_index(drop=True).to_numpy(), index=y_test.index, name="pred"
        )
    )
    .stack()
    .reset_index()
    .rename(columns={"level_2": "variable", 0: "value"})
    .assign(split_type='test')
)
df_pred_naive

CPU times: user 1.49 s, sys: 894 µs, total: 1.49 s
Wall time: 1.49 s


,country,utc_timestamp,variable,value,split_type
0,AT,2019-09-01 00:00:00+00:00,load,4.726,test
1,AT,2019-09-01 00:00:00+00:00,pred,4.697,test
2,AT,2019-09-01 01:00:00+00:00,load,4.592,test
3,AT,2019-09-01 01:00:00+00:00,pred,4.602,test
4,AT,2019-09-01 02:00:00+00:00,load,4.513,test
...,...,...,...,...,...
17275,SK,2019-09-30 21:00:00+00:00,pred,3.079,test
17276,SK,2019-09-30 22:00:00+00:00,load,2.816,test
17277,SK,2019-09-30 22:00:00+00:00,pred,2.971,test
17278,SK,2019-09-30 23:00:00+00:00,load,2.727,test


**Notes**
1. Similar to the predictions made by the ML-based approach, the forecasted and true values from the naive forecasting approach are now arranged into a tidy format.

## Model Evaluation

### Evaluation Metrics

We'll score the ML-trained forecast against the true values from the holdout data (test split)

In [21]:
df_metrics = (
    df_pred.groupby(["country", "split_type"])
    .apply(calculate_evaluation_metrics)
    .reset_index(level=[0, 1])
    .reset_index(drop=True)
    .pivot(index=["country", "split_type"], columns=["metric"], values=["value"])
)
df_metrics.columns = [
    "_".join(cp).replace("value_", "")
    for cp in df_metrics.columns.to_flat_index().tolist()
]
df_metrics = (
    df_metrics.reset_index()
    .set_index("country")
    .sort_index()[["split_type", "rmse", "mae", "smape(%)", "mse", "r2", "rmspe(%)"]]
    .assign(model_type=f"ML_Trained_Using_Last_{num_days_train}d")
)
display(df_metrics.query("split_type == 'test'"))

,split_type,rmse,mae,smape(%),mse,r2,rmspe(%),model_type
country,,,,,,,,
AT,test,0.448632,0.368512,5.657436,0.201271,0.862417,7.349677,ML_Trained_Using_Last_608d
BE,test,0.505355,0.398816,4.451222,0.255383,0.812525,6.097381,ML_Trained_Using_Last_608d
CZ,test,0.362191,0.273367,3.986693,0.131182,0.867310,5.249923,ML_Trained_Using_Last_608d
DE,test,3.584822,2.878979,5.651368,12.850949,0.853394,7.663943,ML_Trained_Using_Last_608d
DK,test,0.212938,0.162995,4.404906,0.045343,0.892550,5.500146,ML_Trained_Using_Last_608d
ES,test,1.941108,1.551036,5.604256,3.767899,0.770616,7.501187,ML_Trained_Using_Last_608d
GB,test,2.551234,1.987502,7.101471,6.508794,0.853389,10.660332,ML_Trained_Using_Last_608d
HR,test,0.124351,0.099406,5.142070,0.015463,0.850246,6.608496,ML_Trained_Using_Last_608d
IT,test,2.807897,2.234474,6.863177,7.884286,0.836307,9.475096,ML_Trained_Using_Last_608d


We'll now score the naive forecast against the same true values from the holdout data (test split)

In [22]:
%%time
df_metrics_naive = (
    df_pred_naive
    .groupby(["country", "split_type"])
    .apply(calculate_evaluation_metrics)
    .reset_index(level=[0, 1])
    .reset_index(drop=True)
    .pivot(index=["country", "split_type"], columns=["metric"], values=["value"])
)
df_metrics_naive.columns = [
    "_".join(cp).replace("value_", "")
    for cp in df_metrics_naive.columns.to_flat_index().tolist()
]
df_metrics_naive = (
    df_metrics_naive.reset_index()
    .set_index("country")
    .sort_index()[["split_type", "rmse", "mae", "smape(%)", "mse", "r2", "rmspe(%)"]]
    .assign(model_type="Naive_Using_Last_Years")
)
df_metrics_naive

CPU times: user 72.7 ms, sys: 361 µs, total: 73.1 ms
Wall time: 72.2 ms


,split_type,rmse,mae,smape(%),mse,r2,rmspe(%),model_type
country,,,,,,,,
AT,test,0.292593,0.247818,3.655782,0.085611,0.941479,4.351534,Naive_Using_Last_Years
BE,test,0.392791,0.315076,3.410727,0.154285,0.886741,4.517279,Naive_Using_Last_Years
CZ,test,0.281829,0.173161,2.507282,0.079428,0.919659,3.801919,Naive_Using_Last_Years
DE,test,2.682452,2.271474,4.233561,7.195551,0.917912,5.096780,Naive_Using_Last_Years
DK,test,0.125997,0.096214,2.749543,0.015875,0.962380,3.542324,Naive_Using_Last_Years
ES,test,1.624165,1.278885,4.475958,2.637912,0.839408,5.805175,Naive_Using_Last_Years
GB,test,2.544515,1.826639,6.527910,6.474556,0.854161,9.908762,Naive_Using_Last_Years
HR,test,0.102261,0.077944,3.925123,0.010457,0.898726,4.960116,Naive_Using_Last_Years
IT,test,1.694897,1.380183,4.114059,2.872677,0.940358,5.062392,Naive_Using_Last_Years


**Notes**
1. Scores are generated per timeseries (i.e. per country).

We'll now combine the metrics for both the ML-based and naive forecasts and reshape the combined output into a tidy format suitable for plotting

In [23]:
df_metrics_all_tidy = (
    pd.concat(
        [
            df.query("split_type == 'test'").reset_index()
            for df in [df_metrics, df_metrics_naive]
        ]
    )
    .sort_values(by=["country", "model_type"])
    .reset_index(drop=True)
    .drop(columns=["split_type"])
    .set_index(["country", "model_type"])
    .stack()
    .reset_index()
    .rename(columns={"level_2": "metric", 0: "value"})
    .assign(model_type=lambda df: df["model_type"].str.replace("_", " "))
    .assign(metric=lambda df: df["metric"].str.upper())
)
df_metrics_all_tidy

,country,model_type,metric,value
0,AT,ML Trained Using Last 608d,RMSE,0.448632
1,AT,ML Trained Using Last 608d,MAE,0.368512
2,AT,ML Trained Using Last 608d,SMAPE(%),5.657436
3,AT,ML Trained Using Last 608d,MSE,0.201271
4,AT,ML Trained Using Last 608d,R2,0.862417
...,...,...,...,...
139,SK,Naive Using Last Years,MAE,0.070492
140,SK,Naive Using Last Years,SMAPE(%),2.278879
141,SK,Naive Using Last Years,MSE,0.008047
142,SK,Naive Using Last Years,R2,0.945167


We'll show the chart of the evaluation metrics with the
- ML-based
- naive

forecast scores shown side-by-side (in a grouped bar chart)

In [24]:
%%time
charts = plot_multi_model_evaluation_metrics(
    df_metrics_all_tidy,
    country_names,
    tooltip=['model_type', 'metric', 'value'],
    ts_separator_name="country",
    ptitle="Evaluation Metrics",
    ptitle_fontsize=14,
    ticklabel_fontsize=12,
    fig_size=dict(width=60, height=200),
)
_ = list(map(display, charts))

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

CPU times: user 595 ms, sys: 4.89 ms, total: 600 ms
Wall time: 590 ms


Finally, the difference between the ML-based and naive forecasts, in [percentage points](https://en.wikipedia.org/wiki/Percentage_point), is charted below for the percentage-based metrics
- R2 (expressed as a percentage)
- RMSPE(%)
- SMAPE(%)

In [25]:
ppt_metrics = ["R2", "RMSPE(%)", "SMAPE(%)"]
df_metrics_ppt_diff = (
    df_metrics_all_tidy.query("metric.isin(@ppt_metrics)")
    .groupby("metric")
    .apply(
        compare_forecasts,
        col_name="model_type",
        col_to_compare="value",
        search_strs=["ML Trained", "Naive"],
    )
    .rename("diff_pct_points")
    .to_frame()
    .T.assign(R2=lambda df: df["R2"] * 100)
    .T.reset_index()
)
df_metrics_ppt_diff

,metric,diff_pct_points
0,R2,-6.924426
1,RMSPE(%),2.183232
2,SMAPE(%),1.492905


In [26]:
plot_bar_chart(
    df_metrics_ppt_diff,
    xvar="metric:N",
    yvar="diff_pct_points:Q",
    ptitle="Percentage Points between ML and Naive Forecasts",
    tooltip=["metric", "diff_pct_points"],
    fig_size=dict(width=450, height=200),
)

alt.LayerChart(...)

**Notes**
1. The difference in [percentage points is calculated as](https://en.wikipedia.org/wiki/Percentage_point)
   - percentage metric from ML-based approach - percentage metric from naive approach

**Observations**
1. The naive forecast outperforms the ML-based forecast in all other metrics.
2. For the following metrics
   - RMSPE (%)
   - SMAPE (%)

   the smaller the absolute value the better. The positive percentage point difference indicates that the ML-based approach
   - produced an absolute value that was larger than that from the naive forecast
   - faired worse than the naive approach

   if scored by these metrics
   - by approximately 2.2 percentage points for RMSPE(%)
   - by approximately 1.5 percentage points for SMAPE(%)
3. For the `R^2` metric, the larger the absolute value the better. The negative percentage point difference indicates that the ML-based approach faired worse than the naive approach by approximately 7 percentage points.

### Model Coefficients

The forecast's (linear model) feature coefficients are now retrieved and will be [scaled by the corresponding features' standard deviation](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html#interpreting-coefficients-scale-matters) before we compare the coefficients

In [27]:
%%time
for c in country_names:
    chart = plot_coef_bar_chart(
        df_coefs_scaled.query(f"country == '{c}'").assign(
            gt_zero=lambda df: df["Coefficient Importance"] > 0
        ),
        xvar="Coefficient Importance",
        yvar="Feature:N",
        ptitle="Feature Coefficients",
        ptitle_x_loc=135,
        tick_label_fontsize=14,
        title_fontsize=16,
        color_by_col="gt_zero:N",
        tooltip=["Feature", "Coefficient Importance"],
        fig_size=dict(width=275, height=1200),
    )
    display(chart)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

...

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

CPU times: user 681 ms, sys: 15.1 ms, total: 696 ms
Wall time: 671 ms


The test split covers the following weeks of the year

In [31]:
[f"Week Of Year={w}" for w in X_test["week_of_year"].unique()]

['Week Of Year=35',
 'Week Of Year=36',
 'Week Of Year=37',
 'Week Of Year=38',
 'Week Of Year=39',
 'Week Of Year=40']

**Observations**
1. The
   - week of year covering the period of the test split (weeks 35 to 40)
   - check if the day is a weekday or weekend
   - check if the day is a holiday
   - hour of the day (RBF features)

   are the most important predictors of electricity consumption in 9 of the 12 countries. Given the highly seasonal patterns expected in usage of electricity, it is expected that these should be strong predictors. For the other three timeseries (DK, ES and CZ), RBF features are the strongest predictors and the others are insignificant. It is not clear why this discrepancy exists in ML model coefficients. Exploratory Data Analysis might help to better understand this observation.

### Model Requirements Diagnostics

Model diagnostics are shown per timeseries (i.e. per country) based on the requirements of [linear regression models](https://faculty.fuqua.duke.edu/~rnau/Decision411_2007/testing.htm).

#### DE

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="DE",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### CZ

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="CZ",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### BE

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="BE",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### HR

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="HR",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### PL

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="PL",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### IT

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="IT",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### SK

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="SK",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### RO

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="RO",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### ES

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="ES",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### AT

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="AT",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### DK

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="DK",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

#### GB_GBN

In [ ]:
plot_diagnostic_charts(
    df_pred,
    c="GB",
    ptitle_hist="Fit Residual",
    yvar="Load",
    ptitle_true_pred_prefix="True vs Predicted Load in",
    tick_label_fontsize=12,
    ptitle_fontsize=14,
    ptitle_qq="Normal Q-Q for Fit Residual of Test Split",
    ptitle_y_vs_x="Predicted vs Observed Test Split",
    n_lags_acf=77,
)

**Observations**

Testing Assumptions of Linear Models
1. Independence of fit residuals (no serial correlation)
   - except for HR, SK, ES and AT, the autocorrelation of fit residuals is usually small (at or below 0.2)
     - this means that there is room for improvement in the model for these four timeseries
   - there is some evidence of cyclical or weak seasonal patterns in the residuals suggesting that seasonality has not been properly accounted for in the model
     - adding seasonal dummies for other `datetime` attributes (eg. week of month, day of week, etc.) should be further explored to handle this
2. linearity of the relationship between dependent and independent variables
   - the `R2` values between predicted and observed values suggest that a linear model is an appropriate choice
   - a linear model is trained against data which are linearly related to the target
   - this means we can be reasonably comfortable of the suitability of the model when we extrapolate beyond the range of the data used here
3. normality of the error distribution
   - the histograms in the diagnostic charts are normally distributed

## Summary

### Benefit of Naive Model for Strongly Seasonal Data
Aggregated electricity consumption at the national level has strong seasonality. This makes a naive forecast a strong benchmark against which to base more sophisticated forecasting approaches.

Using all six evaluation metrics a naive forecast of using electricity consumption from the previous year outperforms the linear model-based ML forecast. The metrics expressed as a percentage show that the **naive forecast is approximately seven percentage points better than the ML-based approach**.

### Model Diagnostics Suggest Model Tuning is Warranted
The check of assumptions of linear models shows some evidence of a pattern in the fit residuals. This suggests the linear ML-model specification should be improved to capture more underlying seasonality in the data.

### Model Explainability
The model coefficients indicate that
- the week of the year covering the period to be forecast (i.e. the weeks covered by the pilot study)
- hour of day
- whether day is a
  - holiday
  - weekday
are the most important features for forecasting electricity consumption at the national level in nine of the 12 countries. In the remaining three countries, the hour of day dominates the other features, but more work is needed to understand why this discrepancy exists.

### Conclusion
A naive forecast which is unable to provide information about factors responsible for the underlying electricity usage. Model explainability is the benefit of a ML approach over the naive approach. This projects there is some tradeoff in
- forecast accuracy
- model explainability

when determining whether a ML approach to forecasting is better than a naive one. The latter is more performant, while **the ML-based approach (though taking a performance hit by approximately seven percentage points) indicates which factors are the strongest predictors** of aggregated electricity consumption by country.

If
- pure performance is the priority, then a naive forecast looking one year into the past is sufficient to forecast consumption during the month of September in 2019 (pilot study period)
- the factors driving electricity consumption are important and can be meaningfully used to efficiently plan maintenance scheduling, then the ML-based approach is recommended
  - these factors could be used to set a higher priority for maintenance operations when electricity usage is forecasted to be the lowest (on holidays, weekends and during night-time/off-peak hours of the day)